<a href="https://colab.research.google.com/github/ishika-b/neural-networks/blob/main/nnfl_assn2_q9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow 
!pip install keras

In [ ]:
from scipy.io import loadmat
import numpy as np

from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

import keras
from keras import models
from keras import layers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Reshape
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils

In [ ]:
input = loadmat('/content/drive/MyDrive/nnfl/nnfl2/input.mat')
labels = loadmat('/content/drive/MyDrive/nnfl/nnfl2/class_label.mat')
data = input['x']
y = labels['y']

x = []
for i in range(17160):
  x.append(data[0][i].T) #(17160, 800, 12)

In [ ]:
print(np.array(x).shape)
print(np.array(y).shape)

(17160, 800, 12)
(17160, 1)


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=1/8, random_state=1)

y_tr_one_hot  = np.zeros((np.array(y_train).shape[0],7))

for i in range(np.array(y_train).shape[0]):
  label = y_train[i][0]-1
  y_tr_one_hot[i][int(label)] = 1


In [ ]:
model = models.Sequential()
model.add(layers.Conv1D(filters = 20, kernel_size= 7, input_shape=(np.array(x_train).shape[1], np.array(x_train).shape[2]) , activation='relu'))
model.add(layers.MaxPooling1D(pool_size=3, strides = 3))
model.add(layers.Conv1D(filters = 60, kernel_size = 7, activation='relu'))
model.add(layers.MaxPooling1D(pool_size=3, strides = 3))
model.add(layers.Dropout(0.7))
model.add(layers.Conv1D(filters = 120, kernel_size= 7))
model.add(layers.Conv1D(filters = 120, kernel_size= 7))
model.add(layers.Flatten())
model.add(layers.Dense(2000, activation = 'relu'))
model.add(layers.Dense(700, activation = 'relu'))
model.add(layers.Dense(50, activation = 'relu'))
model.add(layers.Dense(7, activation = 'sigmoid'))

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 794, 20)           1700      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 264, 20)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 258, 60)           8460      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 86, 60)            0         
_________________________________________________________________
dropout (Dropout)            (None, 86, 60)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 80, 120)           50520     
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 74, 120)           1

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [ ]:
y_te_one_hot  = np.zeros((np.array(y_test).shape[0],7))

for i in range(np.array(y_test).shape[0]):
  label = y_test[i][0]-1
  y_te_one_hot[i][label] = 1

In [ ]:
y_val_one_hot  = np.zeros((np.array(y_val).shape[0],7))

for i in range(np.array(y_val).shape[0]):
  label = y_val[i][0]-1
  y_val_one_hot[i][label] = 1

history_val = model.fit(np.array(x_train), y_tr_one_hot, validation_data=(np.array(x_val), y_val_one_hot), epochs=10, batch_size=1024)

Epoch 1/10
12/12 [==============================] - 54s 5s/step - loss: 0.0124 - accuracy: 0.9944 - val_loss: 3.8417e-04 - val_accuracy: 1.0000
Epoch 2/10
12/12 [==============================] - 54s 4s/step - loss: 0.4025 - accuracy: 0.9313 - val_loss: 0.4530 - val_accuracy: 0.8566
Epoch 3/10
12/12 [==============================] - 53s 4s/step - loss: 0.0975 - accuracy: 0.9774 - val_loss: 0.0010 - val_accuracy: 1.0000
Epoch 4/10
12/12 [==============================] - 54s 5s/step - loss: 0.0016 - accuracy: 1.0000 - val_loss: 1.6108e-04 - val_accuracy: 1.0000
Epoch 5/10
12/12 [==============================] - 54s 5s/step - loss: 0.1085 - accuracy: 0.9819 - val_loss: 1.0254 - val_accuracy: 0.7372
Epoch 6/10
12/12 [==============================] - 54s 4s/step - loss: 0.2077 - accuracy: 0.9490 - val_loss: 0.0010 - val_accuracy: 1.0000
Epoch 7/10
12/12 [==============================] - 54s 5s/step - loss: 0.0018 - accuracy: 0.9998 - val_loss: 1.4067e-04 - val_accuracy: 1.0000
Epoch 8/

In [ ]:
test_loss, test_acc = model.evaluate(np.array(x_test), np.array(y_te_one_hot), verbose=0)
print('overall testing accuracy:',test_acc)

overall testing accuracy: 1.0


In [ ]:
from sklearn.metrics import confusion_matrix
y_pred = np.argmax(model.predict(np.array(x_test)), axis = -1) + 1
#print(y_pred)
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[571   0   0   0   0   0   0]
 [  0 402   0   0   0   0   0]
 [  0   0 524   0   0   0   0]
 [  0   0   0 344   0   0   0]
 [  0   0   0   0 596   0   0]
 [  0   0   0   0   0 625   0]
 [  0   0   0   0   0   0 370]]


In [ ]:
for i in range (7):
  print("accuracy of class ",i+1,":", cm[i,i]/np.sum(cm[i]))

accuracy of class  1 : 1.0
accuracy of class  2 : 1.0
accuracy of class  3 : 1.0
accuracy of class  4 : 1.0
accuracy of class  5 : 1.0
accuracy of class  6 : 1.0
accuracy of class  7 : 1.0
